<a href="https://colab.research.google.com/github/nav-nandan/ccloud-live-lab/blob/master/stream_wars.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Stream Wars

## A simple game in order to get familiar with Confluent Cloud. The team with the maximum sustained ingestion throughput wins.

![](https://drive.google.com/uc?export=view&id=1HW5K7kydLN-Goh-_fSS6WbPJIR6DMsWt)

# Get Confluent Cloud CLI Utilities

In [ ]:
%%bash
curl -O https://packages.confluent.io/archive/7.6/confluent-7.6.2.tar.gz
tar -xzvf confluent-7.6.2.tar.gz

In [ ]:
%%bash
export PATH=$PATH:/content/confluent-7.6.2/bin
confluent --help

Manage your Confluent Cloud or Confluent Platform. Log in to see all available commands.

Usage:
  confluent [command]

Available Commands:
  cloud-signup    Sign up for Confluent Cloud.
  completion      Print shell completion code.
  configuration   Configure the Confluent CLI.
  connect         Manage Kafka Connect.
  context         Manage CLI configuration contexts.
  help            Help about any command
  kafka           Manage Apache Kafka.
  local           Manage a local Confluent Platform development environment.
  login           Log in to Confluent Cloud or Confluent Platform.
  logout          Log out of Confluent Cloud or Confluent Platform.
  plugin          Manage Confluent plugins.
  prompt          Add Confluent CLI context to your terminal prompt.
  secret          Manage secrets for Confluent Platform.
  shell           Start an interactive shell.
  update          Update the Confluent CLI.
  version         Show version of the Confluent CLI.

Flags:
      --versi

# Verify Login to Confluent Cloud

In [ ]:
%%bash
export PATH=$PATH:/content/confluent-7.6.2/bin
export CONFLUENT_CLOUD_EMAIL=<email>
export CONFLUENT_CLOUD_PASSWORD=<password>
confluent login --organization-id <ccloud-org-id> --save
confluent environment list

In [ ]:
%%bash
export PATH=$PATH:/content/confluent-7.6.2/bin
confluent environment use env-0d6g55
confluent kafka cluster list

Using environment "env-0d6g55".
  Current |     ID     |    Name     | Type  | Provider |     Region     | Availability | Network | Status  
----------+------------+-------------+-------+----------+----------------+--------------+---------+---------
  *       | lkc-n03vvk | stream_wars | BASIC | aws      | ap-southeast-1 |              |         | UP      


# Get Confluent Cloud Cluster Endpoint

In [ ]:
%%bash
export PATH=$PATH:/content/confluent-7.6.2/bin
confluent kafka cluster use lkc-n03vvk
confluent kafka cluster list
confluent kafka cluster describe

  Current |     ID     |    Name     | Type  | Provider |     Region     | Availability | Network | Status  
----------+------------+-------------+-------+----------+----------------+--------------+---------+---------
  *       | lkc-n03vvk | stream_wars | BASIC | aws      | ap-southeast-1 |              |         | UP      
+----------------------+--------------------------------------------------------------+
| Current              | true                                                         |
| ID                   | lkc-n03vvk                                                   |
| Name                 | stream_wars                                                  |
| Type                 | BASIC                                                        |
| Ingress Limit (MB/s) | 250                                                          |
| Egress Limit (MB/s)  | 750                                                          |
| Storage              | 5 TB                            

Set Kafka cluster "lkc-n03vvk" as the active cluster for environment "env-0d6g55".


# Create Confluent Cloud API Key

In [ ]:
%%bash
export PATH=$PATH:/content/confluent-7.6.2/bin
confluent api-key create --resource lkc-n03vvk

# Create a Kafka Topic in the Confluent Cloud Cluster
## (Hint: Use topic config to set up topic to be able to stream more data)

In [ ]:
%%bash
export PATH=$PATH:/content/confluent-7.6.2/bin
confluent api-key use <ccloud-api-key> --resource lkc-n03vvk
confluent kafka topic create teamA --partitions 5 --cluster lkc-n03vvk

Using API Key "BGLNYK4WDCZOQM3Q".
Created topic "teamA".


In [ ]:
%%bash
export PATH=$PATH:/content/confluent-7.6.2/bin
confluent kafka topic list --cluster lkc-n03vvk

  Name  | Internal | Replication Factor | Partition Count  
--------+----------+--------------------+------------------
  teamA | false    |                  3 |               5  


# Set Producer Config for use with Confluent Cloud

In [ ]:
%%bash
echo """
client.id=teamA-producer-1
bootstrap.servers=pkc-312o0.ap-southeast-1.aws.confluent.cloud:9092
security.protocol=SASL_SSL
sasl.mechanism=PLAIN
sasl.jaas.config=org.apache.kafka.common.security.plain.PlainLoginModule required username='<ccloud-api-key>' password='<ccloud-api-secret>';
""" > ccloud.properties
cat ccloud.properties


client.id=teamA-producer-1
bootstrap.servers=pkc-312o0.ap-southeast-1.aws.confluent.cloud:9092
security.protocol=SASL_SSL
sasl.mechanism=PLAIN
sasl.jaas.config=org.apache.kafka.common.security.plain.PlainLoginModule required username='<cclouda-api-key>' password='<ccloud-api-secret>';



# Kafka Producer Perf Tool to Stream Data into Confluent Cloud
## (Hint: Get creative with the producer-perf config to stream more data)

In [ ]:
%%bash
export PATH=$PATH:/content/confluent-7.6.2/bin
kafka-producer-perf-test --topic teamA --num-records 1000000 --record-size 100 --throughput 1 --producer.config ccloud.properties

Process is interrupted.
